In [1]:
import camelot
import pandas as pd
import numpy as np
import re
import os

In [2]:
# Helper codes

def clean_country_name(name):
    name = str(name).strip()              
    name = re.sub(r'\s+', ' ', name)     
    name = re.sub(r"\s*-\s*.*", "", name)
    name = re.sub(r"[^A-Za-z\s]", "", name)
    name = re.sub(r'[^A-Za-z\s]', '', name)  
    return name.title()                   

# --- Constants ---
MONTH_ORDER = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

# --- Helpers ---
def get_months(df):
    """Return ordered list of existing month columns."""
    return [m for m in MONTH_ORDER if m in df.columns]


In [3]:
exclude_words = [
    "A S I A",
    "ASEAN",
    "SUB-TOTAL",
    "EAST ASIA",
    "SOUTH ASIA",
    "MIDDLE EAST",
    "A M E R I C A",
    "NORTH AMERICA",
    "SOUTH AMERICA",
    "E U R O P E",
    "WESTERN EUROPE",
    "NORTHERN EUROPE",
    "SOUTHERN EUROPE",
    "EASTERN EUROPE",
    "EASTERN MEDITERRANEAN EUROPE",
    "AUSTRALASIA/PACIFIC",
    "TOTAL (CIS & RUSSIA)"
    "A F R I C A	",
]

In [4]:
# Read tables from PDF (PAGE 1)
pdf_file = "/Users/kim/Desktop/repos/Philippines_Visitor/data/rawData/2020-JAN-DEC.pdf"
tables = camelot.read_pdf(pdf_file, pages='1', flavor="stream")

# Combine all extracted tables into one DataFrame
df1 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df1 = df1.iloc[2:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df1.columns = range(df1.shape[1])

df1.drop(
    df1[df1[0].str.upper().isin(exclude_words)].index,
    inplace=True
)
df1 = df1.drop(columns=[4,6])
df1.columns = range(df1.shape[1])
df1 = df1.iloc[:-1]

df1 = df1.reset_index(drop=True)


In [5]:
# Read tables from PDF (PAGE 2)
tables = camelot.read_pdf(pdf_file, pages='2', flavor="stream")

# Combine all extracted tables into one DataFrame
df2 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df2 = df2.iloc[3:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df2.columns = range(df2.shape[1])

df2.drop(
   df2[df2[0].str.upper().isin(exclude_words)].index,
    inplace=True
)

df2 = df2.drop(columns=[4,5,7,15])
df2.columns = range(df2.shape[1])

df2 = df2.reset_index(drop=True)

# DROP TOTAL RUSSIAN FEDERATION 
df2 = df2.reset_index(drop=True)
df2 = df2.drop(26).reset_index(drop=True)
df2 = df2.drop(23).reset_index(drop=True)

df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,ARGENTINA,711,426,223,3,-,2,-,11,11,12,13,20,"1,432",0.10,"3,643",-60.69
1,BRAZIL,"1,109",940,439,7,1,3,3,22,62,44,56,64,"2,750",0.19,"10,035",-72.60
2,COLOMBIA,334,234,93,2,-,-,2,11,9,10,14,13,722,0.05,"3,386",-78.68
3,PERU,160,123,49,1,-,-,2,4,1,4,2,4,350,0.02,"1,660",-78.92
4,VENEZUELA,80,41,26,,-,-,1,4,10,5,11,7,185,0.01,702,-73.65
5,ANDORRA,5,,3,,-,-,-,-,,-,-,-,8,0.00,25,-68.00
6,AUSTRIA,"1,682","1,609",474,,2,-,8,11,16,16,21,36,"3,875",0.26,"14,840",-73.89
7,BELGIUM,"1,634","1,308",542,5,-,18,24,21,37,44,52,71,"3,756",0.25,"19,156",-80.39
8,FRANCE,"9,405","10,457","4,068",15,2,20,32,95,85,119,81,151,"24,530",1.65,"88,577",-72.31
9,GERMANY,"10,272","10,504","4,248",27,36,21,96,71,101,144,144,229,"25,893",1.75,"103,756",-75.04


In [6]:
# Read tables from PDF (PAGE 3)
tables = camelot.read_pdf(pdf_file, pages='3', flavor="stream")

# Combine all extracted tables into one DataFrame
df3 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df3 = df3.iloc[3:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df3.columns = range(df3.shape[1])

df3.drop(
   df3[df3[0].str.upper().isin(exclude_words)].index,
    inplace=True
)

df3 = df3.drop([6,10]).reset_index(drop=True)
df3 = df3.drop(columns=[4])

df3.columns = range(df3.shape[1])

df3 = df3.iloc[:-16]
df3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,AUSTRALIA,"28,884","19,204","6,553",8,4,9,43,69,102,122,163,169,"55,330",3.73,"286,170",-80.67
1,GUAM,"1,351","1,041",383,-,-,2,9,3,5,30,12,46,"2,882",0.19,"19,835",-85.47
2,NAURU,2,,1,-,-,-,1,-,,-,1,-,5,0.00,93,-94.62
3,NEW ZEALAND,"3,854","2,122",768,3,2,5,11,14,24,23,30,27,"6,883",0.46,"37,872",-81.83
4,PAPUA NEW GUINEA,792,339,198,2,-,10,8,24,11,4,19,27,"1,434",0.10,"8,828",-83.76
5,NIGERIA,416,379,209,1,-,-,1,7,2,8,10,9,"1,042",0.07,"3,439",-69.70
6,SOUTH AFRICA,"1,106",466,260,12,5,2,15,63,44,65,56,65,"2,159",0.15,"8,553",-74.76
7,RESIDENCES,"10,969","7,598","3,290",119,18,66,214,441,601,724,737,813,"25,590",1.73,"86,799",-70.52
8,T O T A L,"782,132","439,852","127,721",927,357,"1,186","3,380","5,364","6,410","8,304","9,069","13,753","1,398,455",94.33,"8,188,477",-82.92
9,OVERSEAS FILIPINOS*,"14,032","22,829","11,604",21,"4,138",336,"2,138","12,966","3,637","1,804","4,535","6,040","84,080",5.67,"72,436",16.07


In [7]:
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

combined_df.columns = [
        "Country",
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
        "Total",
        "Percentage",
        "Previous Total",
        "Growth Rate"
        ]

# Clean and convert column to float
combined_df['Total'] = (
    combined_df['Total']
    .astype(str)               # ensure everything is string
    .str.replace(',', '')      # remove commas
    .str.strip()               # remove leading/trailing spaces
    .replace({'': np.nan, '-': np.nan})  # replace empty strings and '-' with NaN
    .astype(float)             # convert to float
)
combined_df = combined_df.sort_values(by="Total", ascending=False).reset_index(drop=True)

# Copy the row you want to move
row_to_move = combined_df.iloc[5]

# Drop the original row
combined_df = combined_df.drop(5).reset_index(drop=True)

top = combined_df.iloc[:1]   
bottom = combined_df.iloc[1:] 

# Insert the row
combined_df = pd.concat([top, pd.DataFrame([row_to_move]), bottom]).reset_index(drop=True)

combined_df['Country'] = combined_df['Country'].apply(clean_country_name)
combined_df['Country'] = combined_df['Country'].replace('Independent States', 'Commonwealth of Independent States')
combined_df['Country'] = combined_df['Country'].replace('Hongkong', 'Hong Kong')
combined_df['Country'] = combined_df['Country'].replace('Korea', 'South Korea')

combined_df.to_csv("/Users/kim/Desktop/repos/Philippines_Visitor/data/cleanedData/2020cleanData.csv")

In [8]:
# Store Monthly Data
monthly_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/monthly_visitors"

monthly_df = combined_df.iloc[2:].reset_index(drop=True) 
monthly_df['Country'] = monthly_df['Country'].apply(clean_country_name)

months = get_months(combined_df)

monthly_df = monthly_df[['Country'] + months]

mon_out_path = os.path.join(monthly_folder, f"2020_monthly.csv")
monthly_df.to_csv(mon_out_path, index=False)

print(f"Saved 2020 Monthly Visitors csv file")
monthly_df

Saved 2020 Monthly Visitors csv file


,Country,January,February,March,April,May,June,July,August,September,October,November,December
0,South Korea,"206,062","115,943","14,751",79,32,47,202,344,236,323,321,537
1,Usa,"106,444","72,602","20,656",110,56,415,993,954,"1,073","1,843","2,137","4,533"
2,China,"153,109","11,313","4,132",3,2,66,174,84,247,346,448,508
3,Japan,"50,667","60,583","20,424",68,28,60,446,532,611,874,"1,024","1,347"
4,Australia,"28,884","19,204","6,553",8,4,9,43,69,102,122,163,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Laos,88,85,17,1,-,1,3,1,,-,2,5
64,Venezuela,80,41,26,,-,-,1,4,10,5,11,7
65,Luxembourg,77,55,15,,-,-,-,-,,3,2,4
66,Andorra,5,,3,,-,-,-,-,,-,-,-


In [11]:
# Store Special Categories Data

cat_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/special_category"

cat_df = combined_df.head(2)
cat_df['Country'] = cat_df['Country'].replace("T O T A L", "TOTAL")
cat_df['Country'] = cat_df['Country'].replace("Overseas Filipinos***", "Overseas Filipinos")
cat_df["Country"] = cat_df["Country"].astype(str).str.strip().str.title()
cat_df["Country"] = cat_df["Country"].apply(clean_country_name)

cat_df = cat_df.iloc[:, :-4]

# cat_out_path = os.path.join(cat_folder, f"2020_category.csv")
# cat_df.to_csv(cat_out_path, index=False)
cat_df.info()
print(f"Saved 2020 Category csv file")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    2 non-null      object
 1   January    2 non-null      object
 2   February   2 non-null      object
 3   March      2 non-null      object
 4   April      2 non-null      object
 5   May        2 non-null      object
 6   June       2 non-null      object
 7   July       2 non-null      object
 8   August     2 non-null      object
 9   September  2 non-null      object
 10  October    2 non-null      object
 11  November   2 non-null      object
 12  December   2 non-null      object
dtypes: object(13)
memory usage: 336.0+ bytes
Saved 2020 Category csv file


/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_44655/906094395.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df['Country'] = cat_df['Country'].replace("T O T A L", "TOTAL")
/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_44655/906094395.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df['Country'] = cat_df['Country'].replace("Overseas Filipinos***", "Overseas Filipinos")
/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_44655/906094395.py:8: SettingWithCopyW

In [10]:
year_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/yearly_total"

year_df = combined_df[['Country', 'Total', 'Percentage', 'Previous Total', 'Growth Rate']]
year_df = year_df.iloc[2:]

year_df["Previous Total"] = (
    year_df["Previous Total"]
        .astype(str)
        .str.replace('"', '', regex=False)
        .str.replace(',', '', regex=False)   # REMOVE COMMAS
        .str.strip()
)

year_df["Previous Total"] = year_df["Previous Total"].astype(float)

year_df["Growth Rate"] = pd.to_numeric(year_df["Growth Rate"], errors = 'coerce')
year_df["Percentage"] = pd.to_numeric(year_df["Percentage"], errors = 'coerce')

year_path = os.path.join(year_folder, f"2020_year.csv")
year_df.to_csv(year_path, index=False)

print(f"Saved 2020 Year csv file")


Saved 2020 Year csv file
